In [1]:
!export CUBLAS_WORKSPACE_CONFIG=:4096:8
!export CUDA_LAUNCH_BLOCKING=1   # (optional, helps debug determinism)

In [2]:
import os

# Set env vars BEFORE importing huggingface modules
os.environ["HF_HOME"] = "/projects/sciences/computing/sheju347/.cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = "/projects/sciences/computing/sheju347/.cache/huggingface/hub"

# Now import huggingface modules
from huggingface_hub import constants

print("HF_HOME:", constants.HF_HOME)
print("HF_HUB_CACHE:", constants.HF_HUB_CACHE)

HF_HOME: /projects/sciences/computing/sheju347/.cache/huggingface
HF_HUB_CACHE: /projects/sciences/computing/sheju347/.cache/huggingface/hub


In [3]:
from model_trainer import ModelTrainer
from test_performance import TestPerformance, DatasetPath, MMLU_Subset

/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/projects/sciences/computing/sheju347/miniconda3/envs/LLM311/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
import os

trainer = ModelTrainer()

# model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "KrithikV/MedMobile"
# model_name = "Qwen/Qwen2.5-3B-Instruct"
# model_name = os.path.abspath("../../../projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4")
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_entire_UltraMedical_batch_4"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-lr2e-5-bf16"
model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-bf16" # BEST
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base_phi4/11-24-phi4-mini-base-UltraMedical"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_Phi_3_mini"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/fine_tuned_model_no_mask_entire"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-3-fine-tuned-all"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-5-fine-tuned-UltraMedical-MedQA-context/"
# model_name = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-6-fine-tuned-UltraMedical-MedQA-LoRA"

# lora_adapter_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-6-fine-tuned-UltraMedical-MedQA-LoRA"
# lora_adapter_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-10-LoRA-context012-r-16-lr-1e-5"
# lora_adapter_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/10-12-LoRA-Evol-context012-r-16-lr-1e-5-epoch3/checkpoint-37846"
lora_adapter_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/10-17-new-base-model-1e-4-batch8-LoRA-1e-4-r16" # BEST
# lora_adapter_path = "/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/11-10-LoRA-5e-5-r16-context012"



# model_name = "google/flan-t5-xl"

# trainer.load_model(model_name)
trainer.load_model(model_name, lora_adapter_path)

# trainer.load_model_t5(model_name)
# trainer.test_model_MedQA_response()



---------- start checking GPU -----------
GPU: NVIDIA H100 NVL
torch.cuda.is_bf16_supported():  True
---------- finish checking GPU -----------
---------- start loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-bf16 -----------
finish loading tokenizer


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.69it/s]


applying LoRA adapter from: /projects/sciences/computing/sheju347/MedicalQA/train/saved_models/LoRA/10-17-new-base-model-1e-4-batch8-LoRA-1e-4-r16
LoRA adapter applied
finish loading model
torch_dtype: torch.bfloat16
cuda available: True
device: cuda
---------- finish loading model:/projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-bf16 -----------


In [5]:

test = TestPerformance(trainer.model, trainer.tokenizer, trainer.device, is_encoder_decoder = False)

In [11]:
# data_range = [9, 13, 19, 24, 28, 29, 30, 31, 32, 38, 43, 44, 45, 46, 52, 56, 57, 62, 64, 68, 71, 73, 77, 87, 90, 94, 96, 97, 108, 
#               109, 110, 118, 123, 126, 133, 135, 139, 141, 143, 144, 145, 146, 159, 160, 166, 167, 170, 171, 172, 173, 179, 180, 181, 183, 
#               185, 186, 188, 191, 196, 197, 203, 205]

# data_range = range(30, 32) # None #range(34, 35)

data_range = None

# data_range = range(7333, 10178)

# data_range = range(349, 1273)

# data_range = range(889, 1000)

topK_searchEngine = 150
topK_SPLADE = 30
topK_denseEmbedding = 0
topK_crossEncoder = 1
topK_LLM = 0
score_threshold = None #0.7 # 0.8
pick_rag_index = None
get_classifier_training_data = False
use_classifier = False
RRF_models = None #[("SPLADE", 0.8), ("MonoT5", 1), ("Vicuna", 1)] # ["SPLADE", "DenseEmbedding", "MonoT5", "Zephyr", "Vicuna"]

# file_name = "9-30-test_data_150_30_5_H100.txt"
# file_name = "a.txt"
# file_name = "10-14-lora-context012-r16-epoch3-150-30-5-all-data.txt"
# file_name = "10-21-batchsize8-lr1e-4-bf16-new-lora-150-30-5-RRF.txt"
# file_name = "11-4-150-30-5-1-MedQA-ensemble.txt"
# file_name = "11-7-no-rag-ensemble-MedQA-A100.txt"
# file_name = "11-13-Phi3-150-dense30-monoT51.txt"
# file_name = "11-16-Phi3-1000-300-80-5-1.txt"
# file_name = "11-18-Phi3-150-30-1-MonoT5-MedMCQA.txt"
# file_name = "11-25-Phi3-no-rag-ensemble-train.txt"
# file_name = "11-26-Phi4-no-rag-MedQA.txt"
# file_name = "11-26-Phi2.5-3b-no-rag-MedMCQA.txt"
file_name = "12-1-Phi3-150-30-1-MiniLM-MMLU-College_medicine.txt"

# test.SPLIT = "train"
test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.College_medicine, is_ensemble = False, use_RAG = True, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'cais/mmlu' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


use_RAG: True
topK_searchEngine: 150
topK_SPLADE: 30
topK_denseEmbedding: 0
topK_crossEncoder: 1
topK_LLM: 0
score_threshold: None
RRF_models: None
model: /projects/sciences/computing/sheju347/MedicalQA/train/saved_models/base/10-15-UltraMedical-batchsize8-bf16
RAG data: [{'docId': 'pubmed23n0809_15741', 'BM25_score': 93.1505, 'BM25_ranking': 35, 'SPLADE_score': 46.40265655517578, 'SPLADE_ranking': 14, 'crossEncoder_score': np.float32(-2.1287432), 'crossEncoder_ranking': 1}]
outputs text: <|user|> Context:
Aggressive policing and the mental health of young urban men. We surveyed young men on their experiences of police encounters and subsequent mental health. Between September 2012 and March 2013, we conducted a population-based telephone survey of 1261 young men aged 18 to 26 years in New York City. Respondents reported how many times they were approached by New York Police Department officers, what these encounters entailed, any trauma they attributed to the stops, and their overall 

In [ ]:

file_name = "10-22-150-30-5-1-MMLU-College_medicine.txt"

test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.College_medicine, is_ensemble = False, use_RAG = False, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

In [ ]:

file_name = "10-22-150-30-5-1-MMLU-College_biology.txt"

test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.College_biology, is_ensemble = False, use_RAG = False, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

In [ ]:

file_name = "10-22-150-30-5-1-MMLU-Anatomy.txt"

test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.Anatomy, is_ensemble = False, use_RAG = False, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

In [ ]:

file_name = "10-22-150-30-5-1-MMLU-Medical_genetics.txt"

test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.Medical_genetics, is_ensemble = False, use_RAG = False, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

In [ ]:

file_name = "10-22-150-30-5-1-MMLU-Clinical_knowledge.txt"

test.test_accuracy(DatasetPath.MMLU, subset_name = MMLU_Subset.Clinical_knowledge, is_ensemble = False, use_RAG = False, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_denseEmbedding = topK_denseEmbedding, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index, get_classifier_training_data = get_classifier_training_data,
                  use_classifier = use_classifier, RRF_models = RRF_models)

In [ ]:
# data_range = [9, 13, 19, 24, 28, 29, 30, 31, 32, 38, 43, 44, 45, 46, 52, 56, 57, 62, 64, 68, 71, 73, 77, 87, 90, 94, 96, 97, 108, 
#               109, 110, 118, 123, 126, 133, 135, 139, 141, 143, 144, 145, 146, 159, 160, 166, 167, 170, 171, 172, 173, 179, 180, 181, 183, 
#               185, 186, 188, 191, 196, 197, 203, 205]

# data_range = range(4, 5) # None #range(34, 35)

data_range = None

# data_range = range(763, 1273)

topK_searchEngine = 150
topK_SPLADE = 30
topK_crossEncoder = 5
topK_LLM = 0
score_threshold = None # 0.8
pick_rag_index = 4

file_name = "9-5-150_30_5th_RAG_data_H100.txt"
# file_name = "a.txt"

test.test_accuracy(DatasetPath.MedQA, subset_name = None, is_ensemble = False, use_RAG = True, data_range = data_range, file_name = file_name,
                  topK_searchEngine = topK_searchEngine, topK_SPLADE = topK_SPLADE, topK_crossEncoder = topK_crossEncoder, topK_LLM = topK_LLM,
                   score_threshold = score_threshold, pick_rag_index = pick_rag_index)

In [ ]:

model_prompt = """You are a medical question answering assistant.



Question:
A 2-year-old boy is brought to the physician because of decreased appetite and abdominal pain for the last several weeks. Physical examination shows a well-appearing toddler with a palpable left-sided abdominal mass that does not cross the midline. A CT of the abdomen shows a large, necrotic tumor on the left kidney. Histological examination of the kidney mass shows primitive blastemal cells and immature tubules and glomeruli. This tissue is most likely derived from the same embryological structure as which of the following?

[A] : Adrenal medulla
[B] : Thyroid gland
[C] : Papillary muscles
[D] : Anterior pituitary"""

messages = [{"role": "user", "content": f"{model_prompt}"}]

inputs = trainer.tokenizer.apply_chat_template(messages, add_generation_prompt = True, return_tensors = "pt").to(trainer.device)

generate_kwargs = { "max_new_tokens": 1024, "do_sample": False }

outputs = trainer.model.generate(inputs, **generate_kwargs)

text = trainer.tokenizer.batch_decode(outputs)[0]

print(text)

In [ ]:
import re
import ast
import numpy as np

QUESTION_COUNT = 1273

path = ""
file_names = ["9-5-150_30_1_RAG_data_H100.txt", "9-5-150_30_2nd_RAG_data_H100.txt", "9-5-150_30_3rd_RAG_data_H100.txt", 
              "9-5-150_30_4th_RAG_data_H100.txt", "9-5-150_30_5th_RAG_data_H100.txt"]

full_file_names = [path + file_name for file_name in file_names]

class QuestionAnswerData:
    def __init__(self, answer, correct_answer, RAG_data):
        self.answer = answer
        self.correct_answer = correct_answer
        self.is_correct = answer == correct_answer
        # RAG data: [{'docId': 'pubmed23n0827_1593', 'BM25_score': 95.4198, 'BM25_ranking': 44, 'SPLADE_score': 54.33488464355469, 'SPLADE_ranking': 13, 'MonoT5_score': 0.9597430488962883, 'MonoT5_ranking': 1}]
        self.RAG_data = RAG_data

# Get the data

# [ [QuestionAnswerData, QuestionAnswerData, ... (1273 data)], [...], [...], [...], [...] ]
all_question_answer_data_list = []

for i in range(len(full_file_names)):
    question_answer_data_list = []
    with open(full_file_names[i], "r") as f:
        count = 0
        curr_RAG_data = None
        for line in f:
            if "RAG data: " in line:
                curr_RAG_data_str = line[line.find("RAG data: ") + len("RAG data: "):].strip()
                curr_RAG_data = ast.literal_eval(curr_RAG_data_str)

            elif "question " in line:
                
                # Check question number error
                count += 1
                if "num:" in line:
                    number = line.strip().split('num:')[-1].split(' ')[0]
                else:
                    number = line.strip().split('/')[0].split(' ')[-1]
                if int(number) != count:
                    print(f"Error: {file_names[i]}:", number, count)
                    break

                match = re.search(r"answer:([^\s]+)\s+correct_answer:([^\s]+)", line)
                if match:
                    answer = match.group(1)
                    correct_answer = match.group(2)

                    question_answer_data = QuestionAnswerData(answer, correct_answer, curr_RAG_data)
                    question_answer_data_list.append(question_answer_data)
    
    all_question_answer_data_list.append(question_answer_data_list)

print(len(all_question_answer_data_list[0]))
